### 0. Инициализация

In [1]:
# Загружаем библиотеки необходимые для выполнения кода ноутбука.

import pandas as pd
import numpy as np

### 1. Первый этап

#### 1.1 Загрузка первичных данных

Загружаем первичные данные из файлов:
- tracks.parquet
- catalog_names.parquet
- interactions.parquet

In [2]:
tracks = pd.read_parquet('data/tracks.parquet')
cat_names = pd.read_parquet('data/catalog_names.parquet')
interactions = pd.read_parquet('data/interactions.parquet')

### 1.2 Обзор данных

#### 1.2.1 cat_names - справочник каталожных имен

In [3]:
# Типы данных
cat_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1812471 entries, 0 to 1812470
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      int64 
 1   type    object
 2   name    object
dtypes: int64(1), object(2)
memory usage: 41.5+ MB


In [4]:
# Визуальный осмотр данных
cat_names.head(3)

,id,type,name
0,3,album,Taller Children
1,12,album,Wild Young Hearts
2,13,album,Lonesome Crow


In [5]:
# Уникальные типы каталожных единиц
cat_names['type'].value_counts()

type
track     1000000
album      658724
artist     153581
genre         166
Name: count, dtype: int64

In [6]:
# Проверка на пропуски -> отсутствуют
cat_names.isna().sum()

id      0
type    0
name    0
dtype: int64

In [7]:
# Наличие дубликатов по полю 'id' -> присутствуют
cat_names[['id']].duplicated().sum()

35774

In [8]:
# Наличие дубликатов по связке 'id', 'type' -> отсутсвуют
cat_names[['id', 'type']].duplicated().sum()

0

In [9]:
# Наличие дубликатов по связке 'type', 'name' -> присутствуют, и много
cat_names[['type', 'name']].duplicated().sum()

689769

In [10]:
# Наличие дубликатов по связке 'type', 'name' для разных типов
cat_type_name_duplicated = cat_names[cat_names[['type', 'name']].duplicated()]

for type, total_count in cat_names['type'].value_counts().to_dict().items():
    duplicated_count = len(cat_type_name_duplicated.query('type == @type'))
    display(
        f'Type [{type}] : '
        f'duplicated names {duplicated_count} of {total_count} '
        f'or {duplicated_count / total_count * 100:.1f}%.'
    )

del cat_type_name_duplicated, type, duplicated_count, total_count

'Type [track] : duplicated names 469276 of 1000000 or 46.9%.'

'Type [album] : duplicated names 218108 of 658724 or 33.1%.'

'Type [artist] : duplicated names 2385 of 153581 or 1.6%.'

'Type [genre] : duplicated names 0 of 166 or 0.0%.'

__NB! В каталоге около половины треков и трети альбомов имеют повторяющиеся (дублированные) имена.__   
Решение: оставляем как есть, корректировку (слияние) дубликатов в справочниках пока не проводим.

#### 1.2.2 tracks - данные о треках

In [11]:
tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   track_id  1000000 non-null  int64 
 1   albums    1000000 non-null  object
 2   artists   1000000 non-null  object
 3   genres    1000000 non-null  object
dtypes: int64(1), object(3)
memory usage: 30.5+ MB


In [12]:
# Исходное число треков
track_count_initial = len(tracks)

In [13]:
# визуальный отсмотр
tracks.head(3)

,track_id,albums,artists,genres
0,26,"[3, 2490753]",[16],"[11, 21]"
1,38,"[3, 2490753]",[16],"[11, 21]"
2,135,"[12, 214, 2490809]",[84],[11]


In [14]:
# Типы данных для albums, artists, genres
print(*[f'Column "{x}": dtype {type(tracks[x].iloc[0])}\n' 
        for x in ('albums', 'artists', 'genres')])

Column "albums": dtype <class 'numpy.ndarray'>
 Column "artists": dtype <class 'numpy.ndarray'>
 Column "genres": dtype <class 'numpy.ndarray'>



In [15]:
# дубликаты по 'track_id' -> отсутствуют
tracks['track_id'].duplicated().sum()

0

In [16]:
# Все ли track_id есть в справочнике -> да, все
set(tracks['track_id']) - set(cat_names.query('type == "track"')['id'])

set()

In [17]:
# все ли id альбомов есть в справочнике -> нет, не все: 
# встречаются пропуски (nan)
(
    set(tracks['albums'].explode().drop_duplicates()) 
    - set(cat_names.query('type == "album"')['id'])
)

{nan}

In [18]:
# визуальный осмотр битых идентификаторов альбомов
track_ids_with_broken_albums = (
    tracks
    .explode('albums')
    .query('albums.isna()')
    ['track_id']
    .drop_duplicates()
)
print(
    f'Tracks with broken "albums": {len(track_ids_with_broken_albums)} '
    f'or {len(track_ids_with_broken_albums) / track_count_initial :.2%} '
    'of total.\n\nSample:'
)
tracks.query('track_id in @track_ids_with_broken_albums').head(3)

Tracks with broken "albums": 18 or 0.00% of total.

Sample:


,track_id,albums,artists,genres
310821,20200372,[],[],[]
310826,20200380,[],[],[]
312469,20305116,[],[],[]


__Проблема__: в данных о треках `tracks` есть 18 треков с отсутствующими (пустыми) значениями id альбомов.      
_Решение_: удаляем такие треки из справочника (каталога)

In [19]:
# удаляем треки с битыми (пустыми) альбомами
tracks = tracks.query('track_id not in @track_ids_with_broken_albums')

del track_ids_with_broken_albums

In [20]:
# все ли id артистов есть в справочнике -> нет, не все: 
# встречаются пропуски (nan)
(
    set(tracks['artists'].explode().drop_duplicates()) 
    - set(cat_names.query('type == "artist"')['id'])
)

{nan}

In [21]:
# визуальный осмотр битых идентификаторов артистов
track_ids_with_broken_artists = (
    tracks
    .explode('artists')
    .query('artists.isna()')
    ['track_id']
    .drop_duplicates()
)
print(
    f'Tracks with broken "artists": {len(track_ids_with_broken_artists)} '
    f'or {len(track_ids_with_broken_artists) / track_count_initial :.2%} '
    'of total.\n\nSample:'
)
tracks.query('track_id in @track_ids_with_broken_artists').head(3)

Tracks with broken "artists": 15351 or 1.54% of total.

Sample:


,track_id,albums,artists,genres
146031,3599314,[389925],[],"[68, 86]"
146163,3599591,[389944],[],"[68, 86]"
177544,4790215,[533785],[],"[68, 86]"


__Проблема__: в данных о треках `tracks` есть ~1.5% треков с отсутствующими (пустыми) значениями id артистов.      
_Решение_: удаляем такие треки из справочника (каталога)

In [22]:
# удаляем треки с битыми (пустыми) идентификаторами артистов
tracks = tracks.query('track_id not in @track_ids_with_broken_artists')

del track_ids_with_broken_artists

In [23]:
# все ли id жанров есть в справочнике -> нет, не все: 
# встречаются как пропуски (nan) так и безимянные жанры
genre_ids_missing = (
    set(tracks['genres'].explode().drop_duplicates()) 
    - set(cat_names.query('type == "genre"')['id'])
)
print(*genre_ids_missing)

130 131 132 133 134 135 146 148 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 nan 124 126


In [24]:
# визуальный осмотр битых идентификаторов жанров
track_ids_with_broken_genres = (
    tracks
    .explode('genres')
    .query('genres in @genre_ids_missing') 
    ['track_id']
    .drop_duplicates()
)
print(
    f'Tracks with broken "genres": {len(track_ids_with_broken_genres)} '
    f'or {len(track_ids_with_broken_genres) / track_count_initial :.2%} '
    'of total.\n\nSample:'
)
tracks.query('track_id in @track_ids_with_broken_genres').head(3)

Tracks with broken "genres": 51967 or 5.20% of total.

Sample:


,track_id,albums,artists,genres
36,436,[36],[330],"[28, 164]"
59,594,"[54, 88, 5479, 5785124, 9198099, 9231427, 1088...",[533],"[28, 162]"
125,1025,"[94, 2325, 8757, 8986, 318695, 17004129]",[937],"[28, 162]"


__Проблема__: в данных о треках `tracks` есть ~5% треков с неизвестными или отсутствующими (пустыми) значениями id жанров.      
_Решение_: удаляем такие треки из справочника (каталога)

In [25]:
# удаляем треки с неизвестными иои пустыми идентификаторами жанров
tracks = tracks.query('track_id not in @track_ids_with_broken_genres')

del genre_ids_missing, track_ids_with_broken_genres

In [33]:
# сколько треков осталось после чистки данных
print(
    f'Cleaned tracks: {len(tracks)} ' 
    f'or {len(tracks) / track_count_initial:.2%} of inital count.'
)

Cleaned tracks: 932664 or 93.27% of inital count.


In [34]:
# Закодируем 

count    9.326640e+05
mean     3.685616e+07
std      2.670865e+07
min      2.600000e+01
25%      1.573619e+07
50%      3.449496e+07
75%      5.672676e+07
max      1.015218e+08
Name: track_id, dtype: float64

#### 1.2.3 interactions - взаимодействия пользователей с треками

### 1.3 Выводы

_Приведём выводы по первому знакомству с данными:
- есть ли с данными явные проблемы,
- какие корректирующие действия (в целом) были предприняты._

Проблемы с данными в исходных датасетах:
- в каталоге `cat_names` около половины(!) треков и трети альбомов имеют повторяющиеся (дублированные) имена,    
    - _Решение_: оставляем как есть, корректировку (слияние) дубликатов в справочниках пока не проводим.
- в данных о треках `tracks` есть 18 треков с отсутствующими (пустыми) значениями id альбомов,
    - _Решение_: удаляем такие треки из справочника (каталога)
- в данных о треках `tracks` есть ~1.5% треков с отсутствующими (пустыми) значениями id артистов.      
    - _Решение_: удаляем такие треки из справочника (каталога)
- в данных о треках `tracks` есть ~5% треков с неизвестными или отсутствующими (пустыми) значениями id жанров.      
    - _Решение_: удаляем такие треки из справочника (каталога)

# === ЭТАП 2 ===

# EDA

Распределение количества прослушанных треков.

Наиболее популярные треки

Наиболее популярные жанры

Треки, которые никто не прослушал

# Преобразование данных

Преобразуем данные в формат, более пригодный для дальнейшего использования в расчётах рекомендаций.

# Сохранение данных

Сохраним данные в двух файлах в персональном S3-бакете по пути `recsys/data/`:
- `items.parquet` — все данные о музыкальных треках,
- `events.parquet` — все данные о взаимодействиях.

# Очистка памяти

Здесь, может понадобится очистка памяти для высвобождения ресурсов для выполнения кода ниже. 

Приведите соответствующие код, комментарии, например:
- код для удаление более ненужных переменных,
- комментарий, что следует перезапустить kernel, выполнить такие-то начальные секции и продолжить с этапа 3.

# === ЭТАП 3 ===

# Загрузка данных

Если необходимо, то загружаем items.parquet, events.parquet.

# Разбиение данных

Разбиваем данные на тренировочную, тестовую выборки.

# Топ популярных

Рассчитаем рекомендации как топ популярных.

# Персональные

Рассчитаем персональные рекомендации.

# Похожие

Рассчитаем похожие, они позже пригодятся для онлайн-рекомендаций.

# Построение признаков

Построим три признака, можно больше, для ранжирующей модели.

# Ранжирование рекомендаций

Построим ранжирующую модель, чтобы сделать рекомендации более точными. Отранжируем рекомендации.

# Оценка качества

Проверим оценку качества трёх типов рекомендаций: 

- топ популярных,
- персональных, полученных при помощи ALS,
- итоговых
  
по четырем метрикам: recall, precision, coverage, novelty.

# === Выводы, метрики ===

Основные выводы при работе над расчётом рекомендаций, рассчитанные метрики.